In [1]:
import site
site.addsitedir('/home/wojtek/PycharmProjects/mobigen/')

In [2]:
# site.addsitedir('/home/wojtek/PycharmProjects/imputing/')
# from imputing.axiom_vcf_input import *
# pmda_path = "/home/wojtek/Pobrane/Axiom_PMDA.na36.r6.a8.annot.csv/Axiom_PMDA.na36.r6.a8.annot.csv"
# ref_data = read_ref_data(pmda_path)

In [21]:
import time
import re
from urllib.error import *
from Bio import Entrez
from typing import Tuple
import xml.etree.ElementTree as ET
Entrez.email = 'wojciech.galan@gmail.com'
snp_dict = {}#{'rs2336573' :('19', '8302825')}
def get_dbsnp_info(rsid:str) -> Tuple[str, str]:
#     try:
#         return snp_dict[rsid]
#     except KeyError:
    while True:
        try:
            handle = Entrez.efetch(db='snp', id=rsid, report='flt')
            print(handle.url)
            txt = handle.read()
            print("parsing", rsid)
#             print(txt)
            return parse_flt(txt)
        except URLError:
            print(rsid)
            time.sleep(1)
    
def parse_flt(txt:str) -> Tuple[str, str]: # 19:8302825
    root = etree.fromstring(txt)
    return root.find('CHRPOS').text.split(':') #['19', '8302825']

print(get_dbsnp_info('rs2336573')) # ['19', '8302825']

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id=rs2336573&report=flt&tool=biopython&email=wojciech.galan%40gmail.com
parsing rs2336573
['19', '8302825']


In [22]:
from typing import Dict, List, Tuple

def is_int(string:str):
    try:
        int(string)
        return True
    except:
        return False

class RsInfo(object):
    def __init__(self, rsid:str, chrom:str, pos:str):
        self.rsid = rsid
        self.chrom = chrom
        self.pos = pos
    def __lt__(self, other):
        if self.chrom == other.chrom:
            return int(self.pos) < int(other.pos)
        return self.chrom < other.chrom
#         if is_int(self.chrom) and is_int(other.chrom):
#             return int(self.chrom) < int(other.chrom)
#         if is_int(self.chrom) and not is_int(other.chrom):
#             return True
#         if not is_int(self.chrom) and is_int(other.chrom):
#             return False
#         if not is_int(self.chrom) and not is_int(other.chrom):
#             return self.chrom < other.chrom
        
def create_bed_record(rsid:str, chrom:str, pos:str):
    return 'chr{}\t{}\t{}\t{}'.format(chrom, pos, pos, rsid)

def create_bed_records(info:Dict[str, Tuple[str]]) -> List[str]:
    return [create_bed_record(x.rsid, x.chrom, x.pos) for x in sorted([RsInfo(k, *v) for k, v in info.items()])]
    

In [23]:
# odwrotne pozycje w genes.bed
def swap_list_elements(a_list, pos1, pos2):
    temp = str(a_list[pos2])
    a_list[pos2] = str(a_list[pos1])
    a_list[pos1] = temp
with open('genes.bed') as fr:
    with open('genes_reversed.bed','w') as fw:
        for line in fr:
            splitted = line.split()
            swap_list_elements(splitted, 1, 2)
            fw.write('\t'.join(splitted)+'\n')

In [24]:
import os
import importlib
import pickle
for f_name in os.listdir('/home/wojtek/PycharmProjects/mobigen/src/main/resources/traits'):
    base_name = os.path.basename(f_name).split('.')[0]
    bed_name = base_name + '.bed'
    pickle_name = base_name + '.pickle'
    if not (f_name.startswith('_') or f_name.startswith('pusty')) and not (bed_name) in os.listdir('.'):
        print(f_name)
        if pickle_name in os.listdir('.'):
            with open(pickle_name, 'rb') as f:
                records_info = pickle.load(f)
        else:
            module = importlib.import_module('src.main.resources.traits.{}'.format(f_name.split('.')[0])) 
            model = module.model
            rsids = model.get_rsids_used_by_model()
            print(rsids)
            records_info = {rsid:get_dbsnp_info(rsid) for rsid in rsids}
            with open(pickle_name, 'wb') as f:
                pickle.dump(records_info, f)
        bed_records = create_bed_records(records_info)
        with open(bed_name, 'w') as f:
            f.write('\n'.join(bed_records))
            f.write('\n')
        #raise
#from src.main.resources.traits

methylation_model.py
['rs1801133', 'rs2275565', 'rs7422339', 'rs9369898', 'rs7130284', 'rs154657', 'rs234709', 'rs4660306', 'rs548987', 'rs42648', 'rs1801222', 'rs2251468', 'rs838133', 'rs12134663', 'rs12780845', 'rs957140', 'rs12921383', 'rs2851391']
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id=rs1801133&report=flt&tool=biopython&email=wojciech.galan%40gmail.com
parsing rs1801133
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id=rs2275565&report=flt&tool=biopython&email=wojciech.galan%40gmail.com
parsing rs2275565
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id=rs7422339&report=flt&tool=biopython&email=wojciech.galan%40gmail.com
parsing rs7422339
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id=rs9369898&report=flt&tool=biopython&email=wojciech.galan%40gmail.com
parsing rs9369898
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id=rs7130284&report=flt&tool=biopython&email=wojciech.galan%40g

In [25]:
from subprocess import Popen, PIPE
import re, os, math, pickle
from collections import namedtuple
from typing import Tuple, Dict, List
pattern = re.compile('gene_name=([a-zA-Z0-9_.-]+);')

# GeneData = namedtuple('GeneData', 'name start end')

def parse_result(line:str):
    rs_info, gene_info = line.split('|', 1)
    rs_pos = int(rs_info.split()[1])
    splitted_gene_info = gene_info.split()
    dist = min([math.fabs(rs_pos-int(splitted_gene_info[1])), math.fabs(rs_pos-int(splitted_gene_info[2]))])
    gene = pattern.search(gene_info).group(1)
    return rs_info.rsplit(None, 1)[1], gene, int(dist)

def get_closest(rs_file, genes_file) -> Dict[str, Tuple]:
    cmd_pattern = 'closest-features --closest {} {}'
    cmd = cmd_pattern.format(rs_file, genes_file)
    results = {}
    p = Popen(cmd, stdout=PIPE, stderr=PIPE, shell=True)
    output, err = p.communicate()
    for line in output.decode("utf-8").split('\n'):
        if line:
            rs,gene, dist = parse_result(line)
            results[rs] = (gene, dist)
    return results

def choose_closest(rs_list:List[str], closest1:Dict[str, Tuple], closest2:Dict[str, Tuple]):
    for rs in rs_list:
        info1 = closest1.get(rs)
        info2 = closest2.get(rs)
        print(min([info1, info2], key=lambda x: x[1])[0])#, info1, info2)

for f_name in os.listdir('/home/wojtek/PycharmProjects/mobigen/src/main/resources/traits'):
    if not (f_name.startswith('_') or f_name.startswith('pusty')):
        base_name = os.path.basename(f_name).split('.')[0]
        bed_name = base_name + '.bed'
        pickle_name = base_name + '.pickle'
        print(base_name)
        closest = get_closest(bed_name, 'genes.bed')
        closest_rev = get_closest(bed_name, 'genes_reversed.bed')
        with open(pickle_name, 'rb') as f:
            rs_list = list(pickle.load(f).keys())
        choose_closest(rs_list, closest, closest_rev)
#         p = Popen(cmd, stdout=PIPE, stderr=PIPE, shell=True)
#         output, err = p.communicate()
#         for line in output.decode("utf-8").split('\n'):
#             if line:
#                 print(line)
#                 print(pattern.search(line).group(1))
#                 print(parse_result(line))
        print('\n')

b12_model
CD320
TCN2
AC005519.1
AC093864.1
SNRPCP14
TCN1
AC005076.1
FUT2
CUBN
CLYBL
MMUT


met_syn_model
LPL
ZPR1
CETP


prostate_cancer_model
AL445437.1
AC092617.1
MEIS1-AS3
BCL2L11
CDCA7
CASP8
DUBR
MBNL1-AS1
AC074033.1
RNU6-456P
AC008680.1
COL23A1
PTMAP1
HLA-DOA
RF00019
AL136131.2
MIR4655
RF00275
SUGCT
HAUS6
RN7SKP114
TOR1A
LARP4B
RNLS
AL158212.2
MOB2
MIR4487
INCENP
AP001885.1
EMSY
RF00019
AP001007.3
B3GAT1
AC008115.3
GNAI2P1
MIR548C
RNU6-148P
FBRSL1
MMP14
PAX9
RN7SL376P
RFX7
ADGRG1
SCARNA21
AC007923.1
RNF43
MBD2
TCF4-AS2
AC040963.1
TSHZ1
AC020908.3
RNA5SP471
POU2F2
DNMT3B
SUMO1P1
MIR5739
AC002366.1
GOLPH3L
KCNN3
MDM4
NOL10
AC012065.4
RNU6-958P
AC007098.2
RN7SL126P
AC078883.2
MLPH
MIR3133
PPATP1
SIDT1
EEFSEC
ZBTB38
AC073288.2
RNU4ATAC9P
AFM
PDLIM5
PDLIM5
AC004069.1
TERT
AC025183.2
FGF10-AS1
NEDD9
TRIM31-AS1
NOTCH4
HLA-DRA
FOXP4
RNU6-155P
ARMC2
RFX6
RGS17
SLC22A3
AC080068.1
JAZF1
TNS3
LMTK2
SLC25A37
NKX3-1
EBF2
AC024382.1
CASC19
PRNCR1
AC020688.1
POU5F1B
CCAT2
CASC8
RF02045
MARCH8
MSM

AC098484.2
AL663023.1
EPCAM-DT
AC018880.2
RPSAP23
AC067945.1
PCGEM1
HESX1
ENPP7P3
AC110800.1
EHMT2
RF00019
AC006970.3
PRF1
FBXL15
RF02277
AP000852.1
PCDH8
DAPK2
RNU6-19P
RLN3
VPS16
PTPRT


breast_cancer_model
PEX14
PEX14
KLHDC7A
AL391730.1
AC119676.1
AL358075.2
HMGB1P45
PKN2-AS1
AL445437.1
MTND5P20
RF00561
AC244021.1
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
AC244453.1
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
LINC01691
AC243994.1
LINC01691
LINC01691
LINC01691
AL592494.1
SRGAP2-AS1
AC243994.1
PHGDH
RNF115
OTUD7B
DAP3P1
PHLDA3
KRT8P29
ESRRG
EXO1
GRHL1
LINC01376
ADCY3
DNMT3A
LINC01101
INHBB
INHBB
AC012363.2
AC007969.1
DLX2-DT
AC092573.2
CASP8
ALS2CR12
AC007749.1
TNP1
AC007749.1
AC007749.1
AC007749.1
AC007749.1
DIRC3
AC007749.1
AC007749.1
DIRC3
AC007749.1
AC007749.1
DIRC3
AC007563.2
AC007749.1
RN7SKP43
DIRC3
AC062015.1
EGOT
AC018816.2
SLC4A7
RNU6-342P
MICOS10P3
RNU1-96P
AC098614.1
NEK10
AC096921.1
MYL3
M

RF00017
AC073332.1
AC073332.1
RF00092
RF00092
AC012435.2
CLK3
CYP1A1
CYP1A1
NUMBL
CYP2F2P
CYP2F2P
AC008537.3
CYP2A6
CYP2A6
CYP2A6
CYP2A6
CYP2A6
CYP2A6
CYP2A7
CYP2A7P2
CYP2A7P2
CYP2G1P
CYP2B6


blood_pressure_model
CASZ1
MTHFR
AGT
PRPS1P1
RPS20P15
AC074033.1
FGF5
SLC39A8
NPR3
HFE
HOXA4
NOS3
AL450384.2
CABCOCO1
AS3MT
ADRB1
TNNT3
RN7SKP90
KCNJ11
PCNX3
ATP2B1
RF00066
AC009804.1
CPLX3
FURIN
TBX2
RGL3
AL050403.2
ZNF831
PRDM16
SULT1C3
AC116366.2
PHIP
MSH5-SAPCD1
CRIP3
AC073115.2
HRCT1
ADO
CERS5
DPEP1
INSR
ZNRF3


mg_model
MUC1
AC112249.1
TRPM6
DCDC1
BRWD1P2
AC074033.1


folate_model
FOLR3
MTHFR


vitD_model
GC
NADSYN1
CYP2R1
CYP24A1
AMDHD1
AL109628.1


liver_cirrhosis_model
PNPLA3
AC004475.1
TMC4


improved_parkinson_model
POU5F1P4
NUCKS1
ITPKB
SIPA1L2
MAP4K4
VDAC2P4
STK39
MAPRE1P3
SATB1
IP6K2
ITIH1
MCCC1-AS1
TMEM175
AC005798.1
FAM47E-STBD1
AC097478.3
CAMK2D
AC022445.1
AL009179.1
HLA-DQA1
AK3P3
AC025857.2
RN7SL474P
BIN3
SH3GL2
ITGA8
GBF1
RN7SL846P
LDHAL6DP
IGSF9B
RPL30P13
HIP1R
WDHD1
TMEM229B